In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`


In [2]:
#(q, N, L) = (21, 161, 67) 
(q, N, L) = (21, 5, 5) 

(21, 161, 67)

In [3]:
function convert_T3_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, L)
    for i=1:L
        for n=1:N+2
            SCE_ar[n, i] = tmp[n,1,i]
            SCE_ar[N+2+n, i] = tmp[n,2,i]
        end
    end
    return SCE_ar
end
function convert_T5_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L)
    for i=1:L
        for n1=1:N+2
            for n2=1:N+2
                SCE_ar[n1, n2, i] = tmp[n1, 1, n2, 1, i]
                SCE_ar[n1, N+2+n2, i] = tmp[n1, 1, n2, 2, i]
                SCE_ar[N+2+n1, n2, i] = tmp[n1, 2, n2, 1, i]
                SCE_ar[N+2+n1, N+2+n2, i] = tmp[n1, 2, n2, 2, i]
            end
        end
    end
    return SCE_ar
end
function convert_T6_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L, L)
    for i=1:L
        for j=1:L
            for n1=1:N+2
                for n2=1:N+2
                    SCE_ar[n1, n2, i, j] = tmp[n1, 1, n2, 1, i, j]
                    SCE_ar[n1, N+2+n2, i, j] = tmp[n1, 1, n2, 2, i, j]
                    SCE_ar[N+2+n1, n2, i, j] = tmp[n1, 2, n2, 1, i, j]
                    SCE_ar[N+2+n1, N+2+n2, i, j] = tmp[n1, 2, n2, 2, i, j]
                end
            end
        end
    end
    return SCE_ar
end

convert_T6_xnTOy (generic function with 1 method)

In [4]:
T = Float32

Float32

In [5]:
using Random
header = "myseq"
myseq = randstring('A':'Z', N)
ctype=Symbol("amino")
seq = Seq(header, myseq, ctype)

myseq
JNDABAISHNRHERMLLCNHREZHAKTTRTQHYKTHKCTFOKFGXKGBZQHOMXHXGVVDZCMWJPPVIQIUGBGAFHENDQMSFZCNXBLGDVIGDJSWFICUQHRGHIZAXYUIXFZYCLJTCQUUFZNJNPGEFGFQTTQKZSNQSFDNDOZOPQYOG


In [6]:
muint = 1.5
muext = 1.2
lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [7]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=67 N=161 q=21]


In [8]:
damp=T(0.0)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

ParamAlgo{Float32}
-------------
damp=0.0
tol=1.0e-5
tolnorm=1.0e-5
tmax=10
upscheme=sequential
lr=sce
beta=1.0
verbose=true
-------------


In [9]:
bpm = BPMessages(seq, pm)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/bki2w/src/initialization.jl:70


BPMessages{Float32}[L=67 N=161 ongpu=true]


In [10]:
bpb = BPBeliefs(N, L)

LoadError: Out of GPU memory trying to allocate 1.777 GiB
Effective GPU memory usage: 95.69% (22.634 GiB/23.653 GiB)
No memory pool is in use.

In [11]:
lrf = LongRangeFields(N, L)

LongRangeFields{Float32}[L=67 N=161 ongpu=true]

In [12]:
af = AllFields(bpm, bpb, lrf)

LoadError: UndefVarError: bpb not defined

In [17]:
@time BpAlignGpu.update_F!(af, pm, pa)
@time BpAlignGpu.update_hF!(af, pm, pa)
@time BpAlignGpu.update_B!(af, pm, pa)
@time BpAlignGpu.update_hB!(af, pm, pa)
@time BpAlignGpu.update_beliefs!(af, pm)
@time BpAlignGpu.update_jointchain!(af, pm)
@time BpAlignGpu.update_conditional_chain!(af, pa)
@time BpAlignGpu.update_conditional_all!(af, pm)
@time BpAlignGpu.update_f!(af)
@time BpAlignGpu.update_g!(af)

  0.017453 seconds (332 allocations: 19.172 KiB)
  0.000479 seconds (357 allocations: 20.375 KiB)
  0.000491 seconds (332 allocations: 19.281 KiB)
  0.000707 seconds (403 allocations: 23.438 KiB)
  0.000419 seconds (168 allocations: 10.297 KiB)
  0.000336 seconds (204 allocations: 13.844 KiB)
  0.000535 seconds (330 allocations: 19.500 KiB)
  0.001059 seconds (528 allocations: 30.375 KiB)
  0.000274 seconds (139 allocations: 6.266 KiB)
  0.000269 seconds (165 allocations: 8.141 KiB)


In [18]:
@time BpAlignGpu.one_bp_sweep!(af, pm, pa)

  0.003238 seconds (2.96 k allocations: 170.688 KiB)


In [19]:
@time BpAlignGpu.test_sweep!(100,af,pm,pa)

t=1	 err=4.4703484e-7
t=2	 err=3.5762787e-7
t=3	 err=4.7683716e-7
t=4	 err=4.7683716e-7
t=5	 err=4.61936e-7
t=6	 err=3.874302e-7
t=7	 err=3.874302e-7
t=8	 err=6.2584877e-7
t=9	 err=4.172325e-7
t=10	 err=4.4703484e-7
t=11	 err=4.172325e-7
t=12	 err=4.4703484e-7
t=13	 err=3.2782555e-7
t=14	 err=4.7683716e-7
t=15	 err=5.066395e-7
t=16	 err=4.7683716e-7
t=17	 err=2.3841858e-7
t=18	 err=3.874302e-7
t=19	 err=3.7252903e-7
t=20	 err=2.682209e-7
t=21	 err=2.9802322e-7
t=22	 err=5.364418e-7
t=23	 err=2.0861626e-7
t=24	 err=2.0861626e-7
t=25	 err=3.5762787e-7
t=26	 err=6.556511e-7
t=27	 err=4.4703484e-7
t=28	 err=4.7683716e-7
t=29	 err=3.874302e-7
t=30	 err=2.0861626e-7
t=31	 err=2.682209e-7
t=32	 err=2.3841858e-7
t=33	 err=2.3841858e-7
t=34	 err=3.874302e-7
t=35	 err=6.556511e-7
t=36	 err=6.556511e-7
t=37	 err=6.2584877e-7
t=38	 err=4.7683716e-7
t=39	 err=5.662441e-7
t=40	 err=5.364418e-7
t=41	 err=2.9802322e-7
t=42	 err=2.682209e-7
t=43	 err=2.9802322e-7
t=44	 err=3.7252903e-7
t=45	 err=4.4703

# Checks

In [23]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);

In [24]:
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);

In [25]:
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

In [26]:
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);

In [17]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

In [18]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

In [19]:
tmp_Jseq = Array(af.bpm.Jseq)
elts_Jseq = convert_T6_xnTOy(tmp_Jseq);

In [20]:
tmp_Hseq = Array(af.bpm.Hseq)
elts_Hseq = convert_T3_xnTOy(tmp_Hseq);

In [21]:
include("../src/SCE_update_messages.jl")
include("../src/SCE_update_shortrange_beliefs.jl")
include("../src/SCE_update_longrange.jl")

SCE_update_g! (generic function with 1 method)

## check BP updates

In [22]:
BpAlignGpu.update_hF!(af, pm, pa)
BpAlignGpu.update_hB!(af, pm, pa)
BpAlignGpu.update_F!(af, pm, pa)
BpAlignGpu.update_B!(af, pm, pa)

In [27]:
#i=1
#af.bpm.B[:,:,i]

In [28]:
#SCE_update_F!(SCE_F, i, damp, beta, tolnorm, SCE_hF, SCE_f, muext, muint, N, elts_Hseq)
#SCE_update_B!(SCE_B, i, damp, beta, tolnorm, SCE_hB, SCE_f, muext, muint, N, L, elts_Hseq)
#SCE_update_hF!(SCE_hF, i, SCE_F, damp, beta, tolnorm, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)
#SCE_update_hB!(SCE_hB, i, damp, beta, tolnorm, SCE_B, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)

## check update beliefs

In [29]:
#tmp_bel = Array(af.bpb.beliefs)
#SCE_bel = convert_T3_xnTOy(tmp_bel);

In [30]:
#BpAlignGpu.update_beliefs!(af, pm)

In [31]:
#i=1
#SCE_update_beliefs!(SCE_bel, i, beta, tolnorm, SCE_hF, SCE_hB, SCE_f, muint, muext, N, L, elts_Hseq)

In [32]:
#af.bpb.beliefs[:,:,i]

In [33]:
#SCE_bel[:,i]

## check update joint proba

In [34]:
#tmp_joint = Array(af.bpb.joint_chain)
#SCE_joint = convert_T5_xnTOy(tmp_joint);

In [35]:
#sum(SCE_joint)

In [36]:
BpAlignGpu.update_jointchain!(af, pm)

In [37]:
#sum(af.bpb.joint_chain, dims=(1,2,3,4))[1,1,1,1,:]

In [38]:
#i=1
#af.bpb.joint_chain[:,1,:,1,i]

In [39]:
#SCE_update_joint_chain!(SCE_joint, i, beta, tolnorm, SCE_F, SCE_B, SCE_g, lambda_o, lambda_e, N, elts_Jseq);

In [40]:
#rx1 = 1:N+2
#rx2 = N+3:2N+4
#SCE_joint[rx1,rx1, i]

In [41]:
#sum(SCE_joint, dims=(1,2))[1,1,:]

## check update conditional on the chain

In [42]:
BpAlignGpu.update_conditional_chain!(af, pa)

In [43]:
#tmp_joint = Array(af.bpb.joint_chain)
#SCE_joint = convert_T5_xnTOy(tmp_joint);

In [44]:
#cond_chain = fill(0.0, 2N+4, 2N+4, L, L)
#for i=1:L-1
#    SCE_update_conditional_chain!(i, cond_chain, SCE_joint, tolnorm, N)
#end

In [45]:
#af.bpb.conditional[:,2,:,1,1,2]

In [46]:
#cond_chain[rx2, rx1, 1, 2]

In [47]:
#sum(cond_chain[:,:,4,3], dims=1)

In [48]:
#sum(af.bpb.conditional[:,:,:,:,4,3], dims=(1,2))[1,1,:,:]

## check update conditional off the chain

In [49]:
#cond_lr = copy(cond_chain);
#pointer(cond_chain), pointer(cond_lr)

In [50]:
#SCE_update_conditional_lr!(cond_lr)

In [51]:
BpAlignGpu.update_conditional_all!(af, pm);

In [52]:
#cond_lr[rx1, rx2, 4, 1]

In [53]:
#af.bpb.conditional[:,1,:,2,4,1]

In [54]:
#sum(af.bpb.conditional[:,:,:,:,4,1], dims=(1,2))[1,1,:,:]

In [55]:
#sum(cond_lr[:,:,4,1], dims=1)

## check update f

In [58]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [59]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

In [60]:
af.lrf.f[:,:,2]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0
 0.0  0.0

In [61]:
BpAlignGpu.update_f!(af)

In [63]:
using LinearAlgebra
SCE_update_f!(SCE_f, SCE_cond, elts_Jseq, N)

In [64]:
i=3
af.lrf.f[:,:,i]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -4.24892   0.0
 -4.0828   -4.27414
 -3.85625  -4.70392
 -4.40407  -3.8992
 -4.08144  -4.17385
 -1.88375  -3.41813
 -4.19178   0.0

In [65]:
SCE_f[:,i]

14-element Vector{Float64}:
 -4.248915610112336
 -4.082803403479909
 -3.856248602006823
 -4.404074380157429
 -4.081436507314701
 -1.8837479043673235
 -4.191776273157981
  0.0
 -4.274139669206367
 -4.703921061363093
 -3.899199102371751
 -4.17384718815496
 -3.418132984940926
  0.0

## check update g

In [66]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [67]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

In [68]:
af.lrf.g[:,1,:,2,2]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [69]:
BpAlignGpu.update_g!(af)

In [72]:
SCE_update_g!(SCE_g, SCE_cond, elts_Jseq, N)

In [87]:
i=1
af.lrf.g[:,1,:,1,i]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013

In [88]:
rx1 = 1:N+2
rx2 = N+3:2N+4
SCE_g[rx1,rx1,i]

7×7 Matrix{Float64}:
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013
 3.45272  3.45279  3.15216  3.11431  2.71212  2.16102  3.51013